In [2]:
import pandas as pd

df = pd.read_csv('./datasets/job_descriptions.csv')
df = df.sample(frac=0.01)
df.to_csv('./datasets/job_descriptions_sample.csv', index=False)

In [3]:
df = pd.read_csv('./datasets/job_descriptions_sample.csv')
df.dropna(inplace=True)
df.drop(['Job Id', 'latitude', 'longitude', 'Contact Person', 'Contact', 'Company Size', 'Job Posting Date', 'Company Profile', 'Company', 'Job Portal'], axis=1, inplace=True)
df.rename(columns={'skills': 'Skills'}, inplace=True)
df.drop_duplicates(subset=['Job Description', 'Benefits', 'Skills', 'Responsibilities'], inplace=True)
df[['Salary Min', 'Salary Max']] = df['Salary Range'].str.split('-', expand=True)
df[['Salary Min', 'Salary Max']] = df[['Salary Min', 'Salary Max']].apply(lambda x: x.str.replace('$', ''))
df[['Salary Min', 'Salary Max']] = df[['Salary Min', 'Salary Max']].apply(lambda x: x.str.replace('K', ''))
df.drop(['Salary Range'], axis=1, inplace=True)
df[['Experience Min', 'Experience Max']] = df['Experience'].str.split('to', expand=True)
df[['Experience Min', 'Experience Max']] = df[['Experience Min', 'Experience Max']].apply(lambda x: x.str.replace('Years', ''))
df.drop(['Experience'], axis=1, inplace=True)
df['Salary Min'] = pd.to_numeric(df['Salary Min'], errors='coerce')
df['Salary Max'] = pd.to_numeric(df['Salary Max'], errors='coerce')
df.reset_index(drop=True, inplace=True)
df.head(2)

/var/folders/qp/mxd4wgn13yv4m3mf7rxwtml40000gn/T/ipykernel_7373/1738454279.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[['Salary Min', 'Salary Max']] = df[['Salary Min', 'Salary Max']].apply(lambda x: x.str.replace('$', ''))


,Qualifications,location,Country,Work Type,Preference,Job Title,Role,Job Description,Benefits,Skills,Responsibilities,Salary Min,Salary Max,Experience Min,Experience Max
0,BA,Nicosia,Cyprus,Contract,Female,Investment Analyst,Portfolio Manager,A Portfolio Manager oversees investment portfo...,"{'Employee Referral Programs, Financial Counse...",Investment management Financial analysis Risk ...,"Manage investment portfolios, making investmen...",64,91,0,12
1,BBA,San Juan,Puerto Rico,Full-Time,Both,Java Developer,Java Web Application Developer,Java Web Application Developers create web app...,"{'Tuition Reimbursement, Stock Options or Equi...",Web application development Java web framework...,Focus on developing Java-based web application...,57,126,4,14


In [6]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

job_descriptions = df['Job Description'].tolist()
skills = df['Skills'].tolist()

inputs = tokenizer(job_descriptions, padding=True, truncation=True, return_tensors='pt')

encoder = LabelEncoder()
skills_encoded = encoder.fit_transform(skills)

labels = torch.tensor(skills_encoded)

encoder = LabelEncoder()
df['Skills_encoded'] = encoder.fit_transform(df['Skills'])
labels = torch.tensor(df['Skills_encoded'].tolist())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.eval()

with torch.no_grad():
    outputs = model(**inputs, labels=labels)
    predicted_labels = torch.argmax(outputs.logits, dim=1)
    accuracy = (predicted_labels == labels).sum().item() / len(labels)

print("Accuracy:", accuracy)

In [ ]:
model.eval()
job_descriptions_subset = job_descriptions[:10]
inputs_subset = tokenizer(job_descriptions_subset, padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    outputs_subset = model(**inputs_subset)
predicted_labels_subset = torch.argmax(outputs_subset.logits, dim=1)
predicted_skills_subset = encoder.inverse_transform(predicted_labels_subset.tolist())

for i, job_description in enumerate(job_descriptions_subset):
    print("Job Description:", job_description)
    print("Extracted Skills:", predicted_skills_subset[i])
    print()

In [9]:
df_new = pd.read_csv('./datasets/freelance-projects.csv')
df_new.head(3)

,Title,Category Name,Experience,Sub Category Name,Currency,Budget,Location,Freelancer Preferred From,Type,Date Posted,Description,Duration,Client Registration Date,Client City,Client Country,Client Currency,Client Job Title
0,Banner images for web desgin websites,Design,Entry ($),Graphic Design,EUR,60.0,remote,ALL,fixed_price,2023-04-29 18:06:39,We are looking to improve the banner images on...,NaN,2010-11-03,Dublin,Ireland,EUR,PPC Management
1,Make my picture a solid silhouette,"Video, Photo & Image",Entry ($),Image Editing,GBP,20.0,remote,ALL,fixed_price,2023-04-29 17:40:28,Hello \n\nI need a quick designer to make 4 pi...,NaN,2017-02-21,London,United Kingdom,GBP,Office manager
2,Bookkeeper needed,Business,Entry ($),Finance & Accounting,GBP,12.0,remote,ALL,fixed_price,2023-04-29 17:40:06,Hi - I need a bookkeeper to assist with bookke...,NaN,2023-04-09,London,United Kingdom,GBP,Paralegal


In [13]:
new_description = df_new['Description'].tolist()
inputs = tokenizer(new_description, padding=True, truncation=True, return_tensors='pt')

model.eval()
job_descriptions_subset = new_description[:10]


inputs_subset = tokenizer(job_descriptions_subset, padding=True, truncation=True, return_tensors='pt')


with torch.no_grad():
    outputs_subset = model(**inputs_subset)


predicted_labels_subset = torch.argmax(outputs_subset.logits, dim=1)


predicted_skills_subset = encoder.inverse_transform(predicted_labels_subset.tolist())

for i, job_description in enumerate(job_descriptions_subset):
    print("Job Description:", job_description)
    print("Extracted Skills:", predicted_skills_subset[i])
    print('-----------------------------------')


Job Description: We are looking to improve the banner images on our web design pages 

Web design
Shopify 
 woo commence 
redesign
E-commerce

www.leapdigital.ie 

I am attaching two images that I like you will have to create the images in a similar design 


Extracted Skills: Account management Client relations Marketing strategies Campaign optimization Data analysis Communication skills
-----------------------------------
Job Description: Hello 

I need a quick designer to make 4 pictures a solid silhouette with white trimmings around some parts. This is needed to be transferred to a towel to be embroidered. This hopefully should be a quick thing. Solid black and another with white outline
Extracted Skills: API design and development RESTful API knowledge Security protocols (OAuth, JWT)
-----------------------------------
Job Description: Hi - I need a bookkeeper to assist with bookkeeping. Thanks.  
Extracted Skills: Account management Client relations Marketing strategies Campaign 